In [2]:
from io import StringIO 
import pandas as pd
import requests

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filings = pd.read_html(StringIO(response.text))

In [4]:
ipos_df = ipo_filings[0]
ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     329 non-null    object
 1   Symbol          329 non-null    object
 2   Company Name    329 non-null    object
 3   Price Range     329 non-null    object
 4   Shares Offered  329 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


In [5]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
1,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
2,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
3,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,"Apr 16, 2024",JLJT,Jialiang Holdings Ltd,$5.00,-


In [6]:
# get date
ipos_df['Filing Date'] = pd.to_datetime(ipos_df['Filing Date'])

In [7]:
ipos_df['Shares Offered'] = pd.to_numeric(ipos_df['Shares Offered'].astype(str).str.replace('-', ''))

In [8]:
ipos_df['Price Range'] = ipos_df['Price Range'].astype(str).str.replace('$', '').str.replace('-', '')

In [9]:
def get_number_avg(string_range):
    numeric_range = [float(number) for number in string_range.split()]
    not_nubmer = len(numeric_range) < 1 
    if not_nubmer:
        return ''
    return sum(numeric_range) / len(numeric_range)

In [10]:
ipos_df['Avg_price'] = pd.to_numeric(ipos_df['Price Range'].map(get_number_avg))

In [11]:
ipos_df['Shares_offered_value'] = ipos_df['Shares Offered'] * ipos_df['Avg_price'] 

In [12]:
ipos_filtered = ipos_df[(ipos_df['Filing Date'] > '2023-01-01') & (ipos_df['Filing Date'] < '2023-12-31') & (ipos_df['Filing Date'].dt.dayofweek == 4)]

In [13]:
ipos_filtered.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
52,2023-12-29,LEC,Lafayette Energy Corp,3.50 4.50,1200000.0,4.0,4800000.0
53,2023-12-29,EPSM,Epsium Enterprise Limited,,NaN,NaN,NaN
56,2023-12-22,LZMH,LZ Technology Holdings Limited,,NaN,NaN,NaN
57,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,,NaN,NaN,NaN
62,2023-12-15,GIT,Going International Holding Company Limited,,NaN,NaN,NaN


In [14]:
ipos_filtered['Shares_offered_value'].sum()

275950000.0